In [1]:
"""
Authors: Ritvik Kapila, Gauri Gupta
"""

# Updates from previous version:
# 1. Added the implementation for counter_for_l where we need to specify the list of group by columns for only the included sheets in correct order.
# 2. Added the configuration for excluding particular sheets where all the remaining sheets have the same number of group by columns.
# 3. Handled the exceptions occurring due to hidden sheets. Hidden sheets are now ignored and do not affect the output csv. (Sheet.visibility)
# 4. Handled the exception of reading an integer as a float by type casting it as an int if the fractional part is 0.

'\nAuthors: Ritvik Kapila, Gauri Gupta\n'

In [2]:
import xlrd 
import pandas as pd
import math

In [3]:
def xlsx_to_csv_include(input_filename, list_of_number_of_group_by_columns, list_of_sheet_names, header_row_index, data_row_begin_index, output_filename):
    wb = xlrd.open_workbook(input_filename) 
    
    l = list_of_number_of_group_by_columns
    
#     If the list_of_sheet_names is entered as empty list, we include all the sheets in the csv
    
    if(len(list_of_sheet_names) == 0):
        list_of_sheet_names = wb.sheet_names()
    
#     Forming a dictionary 'col' having keys as column names and values as the respective unique number column_counter
#     This helps in appending rows by changing values in an empty list of length n = len(col.keys()) at only certain indices.

    col = {}
    col['Class'] = 0
    column_counter = 1
    
# We add a counter for the list to make a seperate iterator from the iterator for number of sheets in the workbook

    counter_for_l = 0
    
    for i in range(len(wb.sheet_names())):
        if(list_of_sheet_names.count(wb.sheet_names()[i]) > 0):
            sheet = wb.sheet_by_index(i)
            
            for j in range(l[counter_for_l]):
                try:
                    if(col[sheet.cell_value(header_row_index,j)]):
                        continue
                except KeyError:
                    col[sheet.cell_value(header_row_index,j)] = column_counter
                    column_counter += 1
                    
            counter_for_l += 1
            
    col['Parameter'] = column_counter
    column_counter += 1
    col['Value'] = column_counter
    n = len(col.keys())

#     Initializing an empty list of length n and changing the values corresponding to that particular sheet
#     Changing values of the group by columns and adding parameter and value for all non-group by columns
    
    counter_for_l = 0
    
    total_rows = []
    for i in range(len(wb.sheet_names())):
        if(list_of_sheet_names.count(wb.sheet_names()[i]) > 0):    
            sheet = wb.sheet_by_index(i)

            for j in range(data_row_begin_index, sheet.nrows):

                group_by_col_list = {}
                for k in range(l[counter_for_l]):
                    
# If the sheet.cell_value has a zero fractional part, it is forced to be stored as an integer
                    
                    if(sheet.cell_type(j, k) == 2):
                        if(math.modf(sheet.cell_value(j, k))[0] == 0):
                            group_by_col_list[sheet.cell_value(header_row_index, k)] = int(sheet.cell_value(j, k))
                    else:
                        group_by_col_list[sheet.cell_value(header_row_index, k)] = sheet.cell_value(j, k)

                group_by_row = [''] * n
                for k in group_by_col_list:
                    group_by_row[col[k]] = group_by_col_list[k]

                for k in range(l[counter_for_l], sheet.ncols):
                    row = group_by_row.copy()
                    row[col['Class']] = wb.sheet_names()[i]
                    row[col['Parameter']] = sheet.cell_value(header_row_index, k)
                    
# If the sheet.cell_value has a zero fractional part, it is forced to be stored as an integer

                    if(sheet.cell_type(j, k) == 2):
                        if(math.modf(sheet.cell_value(j, k))[0] == 0):
                            row[col['Value']] = int(sheet.cell_value(j, k))
                    else:
                        row[col['Value']] = sheet.cell_value(j, k)
                    
                    total_rows += [row]
                    
            counter_for_l += 1
#     Forming a column list from the dictionary keys

    columns = [''] * n
    for key in col:
        columns[col[key]] = key
        
#     Output file in the form of a csv

    df = pd.DataFrame(total_rows, columns = columns)
    print(df)
    df.to_csv(output_filename, index = False)



In [4]:
def xlsx_to_csv_exclude(input_filename, number_of_group_by_columns, list_of_sheet_names_to_exclude, header_row_index, data_row_begin_index, output_filename):
    wb = xlrd.open_workbook(input_filename) 

# Initializing list l of number_of_group_by_columns which is a number and would be same for all sheets. Here we have the list where the length includes both hidden and visible sheets after excluding certain sheets, but the hidden sheets are ignored afterwards.
# len(l) includes hidden sheets + visible sheets - excluded sheets, hence it won't be parsed completely when there are hidden sheets, but it doesn't matter because the values are all equal.

    l = [number_of_group_by_columns] * (len(wb.sheet_names()) - len(list_of_sheet_names_to_exclude))
    
#     Forming a dictionary 'col' having keys as column names and values as the respective unique number column_counter
#     This helps in appending rows by changing values in an empty list of length n = len(col.keys()) at only certain indices.

    col = {}
    col['Class'] = 0
    column_counter = 1
    
# We add a counter for the list to make a seperate iterator from the iterator for number of sheets in the workbook

    counter_for_l = 0
    
    for i in range(len(wb.sheet_names())):
        if(list_of_sheet_names_to_exclude.count(wb.sheet_names()[i]) == 0):
            sheet = wb.sheet_by_index(i)

            if(sheet.visibility == 0):
                for j in range(l[counter_for_l]):
                    try:
                        if(col[sheet.cell_value(header_row_index,j)]):
                            continue
                    except KeyError:
                        col[sheet.cell_value(header_row_index,j)] = column_counter
                        column_counter += 1

                counter_for_l += 1
            
    col['Parameter'] = column_counter
    column_counter += 1
    col['Value'] = column_counter
    n = len(col.keys())

#     Initializing an empty list of length n and changing the values corresponding to that particular sheet
#     Changing values of the group by columns and adding parameter and value for all non-group by columns
    
    counter_for_l = 0
    
    total_rows = []
    for i in range(len(wb.sheet_names())):
        if(list_of_sheet_names_to_exclude.count(wb.sheet_names()[i]) == 0):    
            sheet = wb.sheet_by_index(i)

            if(sheet.visibility == 0):
                for j in range(data_row_begin_index, sheet.nrows):

                    group_by_col_list = {}
                    for k in range(l[counter_for_l]):
                        
# If the sheet.cell_value has a zero fractional part, it is forced to be stored as an integer

                        if(sheet.cell_type(j, k) == 2):
                            if(math.modf(sheet.cell_value(j, k))[0] == 0):
                                group_by_col_list[sheet.cell_value(header_row_index, k)] = int(sheet.cell_value(j, k))
                        else:
                            group_by_col_list[sheet.cell_value(header_row_index, k)] = sheet.cell_value(j, k)

                    group_by_row = [''] * n
                    for k in group_by_col_list:
                        group_by_row[col[k]] = group_by_col_list[k]

                    for k in range(l[counter_for_l], sheet.ncols):
                        row = group_by_row.copy()
                        row[col['Class']] = wb.sheet_names()[i]
                        row[col['Parameter']] = sheet.cell_value(header_row_index, k)
                        
# If the sheet.cell_value has a zero fractional part, it is forced to be stored as an integer

                        if(sheet.cell_type(j, k) == 2):
                            if(math.modf(sheet.cell_value(j, k))[0] == 0):
                                row[col['Value']] = int(sheet.cell_value(j, k))
                        else:
                            row[col['Value']] = sheet.cell_value(j, k)
                        total_rows += [row]

                counter_for_l += 1
            
#     Forming a column list from the dictionary keys

    columns = [''] * n
    for key in col:
        columns[col[key]] = key
        
#     Output file in the form of a csv

    df = pd.DataFrame(total_rows, columns = columns)
    print(df)
    df.to_csv(output_filename, index = False)



In [5]:
def xlsx_to_csv_main(input_filename, number_of_group_by_columns, list_of_sheet_names, header_row_index, data_row_begin_index, output_filename, includeflag):
    if(includeflag == 1):
        xlsx_to_csv_include(input_filename, number_of_group_by_columns, list_of_sheet_names, header_row_index, data_row_begin_index, output_filename)
    else:
        xlsx_to_csv_exclude(input_filename, number_of_group_by_columns, list_of_sheet_names, header_row_index, data_row_begin_index, output_filename)

In [9]:
# includeflag = 1 for using implementation for including particular sheets
# includeflag = 0 for using implementation for excluding particular sheets where all the remaining sheets have same number of group by columns

# number_of_group_by_columns for includeflag = 1 is the list of number of initial columns which are taken as group by columns.
# If includeflag = 1, number_of_group_by_columns will be a list eg. [5,6,3,2,5,3]

# number_of_group_by_columns for includeflag = 0 is the number of group by columns for all the sheets except the excluded sheets
# If includeflag = 0, number_of_group_by_columns will be a number eg. 3

# configurable_list_of_sheet_names for includeflag = 1 is the list of names of the sheets that we want to include in our csv file. 
# configurable_list_of_sheet_names for includeflag = 1 can be left empty in order to include all the sheets.

# configurable_list_of_sheet_names for includeflag = 0 is the list of names of the sheets that we want to exclude from our csv file. 

# header_row_index is the index of the header rows or column names.
# data_row_begin_index is the index of the row from which the data begins.

# Hidden sheets have been taken care of and are not included in the output csv file

number_of_group_by_columns = [4,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3]


configurable_list_of_sheet_names = ['GCELL', 'GCELLAMRQUL', 'GCELLBASICPARA', 'GCELLCCACCESS', 'GCELLCCAD', 'GCELLCCAMR', 'GCELLCCBASIC', 'GCELLCCCH', 'GCELLCCTMR', 'GCELLCCUTRANSYS', 'GCELLPRIEUTRANSYS', 'GCELLCHMGAD', 'GCELLCHMGBASIC', 'GCELLDYNTURNOFF', 'GCELLEGPRSPARA', 'GCELLEXTMSRPARA', 'GCELLGPRS', 'GCELLGSMR', 'GCELLHOAD', 'GCELLHOBASIC', 'GCELLHOCTRL', 'GCELLHOEDBPARA', 'GCELLHOEMG', 'GCELLHOFAST', 'GCELLHOFITPEN', 'GCELLHOINTERRATLDB', 'GCELLHOIUO', 'GCELLHOUTRANFDD', 'GCELLHOUTRANTDD', 'GCELLIDLEAD', 'GCELLIDLEBASIC', 'GCELLLCS', 'GCELLOPTREV', 'GCELLNC2PARA', 'GCELLNCRESELECTPARA', 'GCELLNWCTRLMSRPARA', 'GCELLFREQSCAN', 'GCELLOSPMAP', 'GCELLOTHBASIC', 'MDTLCS', 'GCELLMOCNINTERRESEL', 'GCELLOPEUTRANNARFCN', 'GCELLOPUTRANNARFCN', 'GCELLADPLMNPARA', 'GCELLMOCNINTERRAT', 'GCELLMOCN', 'GCELLOTHEXT', 'GCELLPRACH', 'GCELLPRIVATEOPTPARA', 'GCELLPSBASE', 'GCELLPSCHM', 'GCELLPSCS', 'GCELLPSI1', 'GCELLPSOTHERPARA', 'GCELLPSPWPARA', 'GCELLPWR2', 'GCELLPWR3', 'GCSCHRSCOPE', 'GMRSCOPE', 'GPSCHRSCOPE', 'GCELLOTHPARA', 'PTPBVC', 'GCELLPWRBASIC', 'GCELLRESELECTPARA', 'GCELLSBC', 'GCELLSERVPARA', 'GCELLSOFT', 'GCELLSTANDARDOPTPARA', 'GCELLTMR', 'GCELLVAMOS', 'GCELLVAMOSPWR', 'GCELLUNDPARA', 'GCELLRESELECTUTRANTDD', 'GCELLPSABISPARA', 'BTSCELLPATCHPARA', 'GCELLBTSSOFTPARA', 'GCELLNONSTANDARDBW', 'GCELLPSDIFFSERVICE', 'GCELL3GARFCN', 'GCELLRESELUTRANFDD', 'GCELLRSVPARA', 'GCELLTRANPARA', 'GCELLPSSMALLPKTRESBAL', 'GCELLWLAN', 'GCELLGLSPECCLOUD', 'GCELLSON', 'GCELLHSRPLCUSRIDFMG', 'GCELLCSFBPARA', 'GCELLCONGACALGO', 'GCELLSRVCC', 'CELLGLDSS', 'GCELLTA', 'GCELLIBCAII']


header_row_index = 0
data_row_begin_index = 2
includeflag = 1

xlsx_to_csv_main('CUS__3__2G Cell Parameters Data Template.xlsx', number_of_group_by_columns, configurable_list_of_sheet_names, header_row_index, data_row_begin_index, 'Result_Sheet.csv', includeflag)

               Class  BSCName          BTSNAME CELLNAME CELLID NSEI BTSID  \
0              GCELL  RWBSC01  Kizungu(WE2380)  WE2380A      0              
1              GCELL  RWBSC01  Kizungu(WE2380)  WE2380A      0              
2              GCELL  RWBSC01  Kizungu(WE2380)  WE2380A      0              
3              GCELL  RWBSC01  Kizungu(WE2380)  WE2380A      0              
4              GCELL  RWBSC01  Kizungu(WE2380)  WE2380A      0              
...              ...      ...              ...      ...    ...  ...   ...   
4714054  GCELLIBCAII  BUBSC03                   KA1698C    620              
4714055  GCELLIBCAII  BUBSC03                   KA1698C    620              
4714056  GCELLIBCAII  BUBSC03                   KA1698C    620              
4714057  GCELLIBCAII  BUBSC03                   KA1698C    620              
4714058  GCELLIBCAII  BUBSC03                   KA1698C    620              

                       Parameter           Value  
0                       